In [ ]:
!pip install pip install keras

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2
from keras import Sequential
from keras.layers import Flatten,Dense,BatchNormalization,Dropout,LeakyReLU,GlobalAveragePooling2D
from keras.optimizers import Adam,RMSprop
from keras.losses import BinaryCrossentropy
from keras.applications import ResNet50V2
from keras.callbacks import EarlyStopping
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, callbacks, utils, applications, optimizers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
data_path = '/content/drive/MyDrive/Colab Notebooks/fatty liver/data/'
data_dir_list = os.listdir(data_path)
test_path ='/content/drive/MyDrive/Colab Notebooks/fatty liver/val/'


In [ ]:
normal = glob.glob(data_path+"normal/*.jpg")
print(len(normal))
g1 = glob.glob(data_path+"g1/*.jpg")
print(len(g1))
g2 = glob.glob(data_path+"g2/*.jpg")
print(len(g2))

259
184
86


In [ ]:
train_val_generator = ImageDataGenerator(dtype='float32', preprocessing_function=preprocess_input)
test_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
test_datagen = ImageDataGenerator(dtype='float32', preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(test_path,
                                                   batch_size = 128,
                                                   target_size = (220,220),class_mode = 'categorical')

Found 36 images belonging to 3 classes.


In [ ]:
train = train_val_generator.flow_from_directory(data_path,
                                               batch_size=128,
                                               target_size=(220,220),
                                               color_mode="rgb",
                                              class_mode="categorical",
                                               shuffle=True,
                                               seed=42,
                                               subset="training")

Found 529 images belonging to 3 classes.


In [ ]:
resnet50 = ResNet50V2(weights = "imagenet", input_shape = (220,220,3), include_top = False)

94668760/94668760 [==============================] - 0s 0us/step


In [ ]:
for layer in resnet50.layers:
    layer.trainable = False
model = Sequential()
res_model = ResNet50(include_top=False, pooling='avg', weights='imagenet', input_shape = (220, 220, 3))
model.add(res_model)

for layer in resnet50.layers:
    layer.trainable = False

model.add(Flatten())

model.add(Dense(units = 128, activation = "relu"))
model.add(Dropout(0.5))

model.add(Dense(units = 3, activation = "softmax"))


94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
Total params: 23,850,371
Trainable params: 23,797,251
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
rlp = callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=5, min_lr=0.0001, mode='min', verbose=2)
early_stopping = EarlyStopping(monitor='loss', patience=5, verbose=2, mode='min', restore_best_weights=True)
hist = model.fit(train,epochs=50,callbacks = [ rlp,early_stopping])

Epoch 1/50
5/5 [==============================] - 8s 1s/step - loss: 0.0013 - accuracy: 1.0000 - lr: 1.0000e-04
Epoch 2/50
5/5 [==============================] - 7s 1s/step - loss: 0.0021 - accuracy: 1.0000 - lr: 1.0000e-04
Epoch 3/50
5/5 [==============================] - 8s 2s/step - loss: 0.0016 - accuracy: 1.0000 - lr: 1.0000e-04
Epoch 4/50
5/5 [==============================] - 6s 2s/step - loss: 0.0116 - accuracy: 0.9962 - lr: 1.0000e-04
Epoch 5/50
5/5 [==============================] - 9s 1s/step - loss: 0.0019 - accuracy: 1.0000 - lr: 1.0000e-04
Epoch 6/50
5/5 [==============================] - ETA: 0s - loss: 0.0018 - accuracy: 1.0000
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0001.
Restoring model weights from the end of the best epoch: 1.
5/5 [==============================] - 9s 2s/step - loss: 0.0018 - accuracy: 1.0000 - lr: 1.0000e-04
Epoch 6: early stopping


In [ ]:
result = model.evaluate(test_generator, steps=3)

1/3 [=========>....................] - ETA: 0s - loss: 0.3852 - accuracy: 0.9444

3/3 [==============================] - 0s 17ms/step - loss: 0.3852 - accuracy: 0.9444
